In [ ]:
# 출처 : https://wikidocs.net/159467
# 모델검색 : https://huggingface.co/models

### 1.Import(관련내용 설치)

In [28]:
!pip install sentence_transformers -qqq
!pip install konlpy -qqq

In [ ]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

from datetime import datetime, timedelta

In [ ]:
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += " " + ele.strip()
    return str1


# Max Sum Similarity : 키워드 추출 시 문서와의 유사성을 극대화
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]


# Maximal Marginal Relevance(MMR) : 텍스트 요약 시, 중복 최소화를 통해 결과의 다양성을 극대화하는 방법
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

### 2.Data Load & Text Cleansing

In [29]:
# Data Load
df = pd.read_excel('./20230616_selex_review_data.xlsx', 1)

In [30]:
# 자료 수 확인
print('- 자료 수 : {}'.format(len(df)) )
# print('- 최종날짜 : {}'.format(max(df.REG_DTM) ) )
# print('- 대상제품 : {}'.format(list(df['PRODUCT'].unique())) )

- 자료 수 : 253


In [31]:
df.tail(1)

,Unnamed: 0,USER,REG_DTM,CHANNEL,PRODUCT,REVIEW,SCORE,KEYWORD,GROUP1,GROUP2
252,7951,dune91,2023-06-09,H26 셀렉스 몰,프로핏 스파클링 청포도,탄산음료 좋아하지만 웨이프로틴 스파클링은 맛도있고 조금이지만 단백질도 함유하고있어...,5,단백,제품,맛(미각)


In [32]:
# Text Cleansing
try:
  doc = listToString(df.REVIEW.sample(frac = 1))
except:
  doc = listToString(df.리뷰상세내용.sample(frac = 1))
  pass

In [33]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') | (word[1] == 'Verb') | (word[1] == 'Adjective')])
# tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') ])

# print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
# print('명사 추출 :',tokenized_nouns)

In [34]:
# 2음절 ~ 3음절
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 10개만 출력 :',candidates[0:9])

trigram 개수 : 5704
trigram 10개만 출력 : ['가게 되니' '가게 되니 기분' '가게 되더라구요' '가게 되더라구요 요건' '가격 구입' '가격 구입 있게' '가격 대만'
 '가격 대만 좀더' '가격 부담']


### 3.모형2 : xlm-r-100langs-bert-base-nli-stsb-mean-tokens

In [35]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens') # OK / 10분?

In [36]:
# 문서간 유사도
doc_embedding = model.encode([doc])

# 단어간 유사도 : 30분 초과시 pass
candidate_embeddings = model.encode(candidates)

In [37]:
top_n = 15
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords3 = mmr(doc_embedding, candidate_embeddings, candidates, top_n=15, diversity=0.8)
keywords2 =  max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=15, nr_candidates=20)

df_k = pd.DataFrame()
df_k['BERT'] =  keywords
df_k['BERT_MMS'] =  keywords2
df_k['BERT_MMR'] =  keywords3

In [38]:
df_k

,BERT,BERT_MMS,BERT_MMR
0,칼로리 적당한 단맛,않은 요구르트,우유 레몬에이드 맛있어요
1,맛있어요 탄산음료,슈거 런가 짠맛,이유 매일유업
2,밀크 들어간 레몬,하지 않은 요구르트,다른 단백질 바보
3,바삭바삭하니 정말 맛있어요,칼로리 적당한 단맛,수가 없던
4,요구르트 맛있게,맛있어요 탄산음료,가루 유산균 먹다
5,레몬에이드 맛있어요,밀크 들어간 레몬,다른 제품 구매
6,좋아하는 밀크 사탕,바삭바삭하니 정말 맛있어요,걱정 건강 당뇨
7,청포도 취향,레몬에이드 맛있어요,아침 먹이 있어요
8,초콜릿우유 즐겨,좋아하는 밀크 사탕,여름철 시원하게 단백질
9,초콜릿우유 즐겨 마셔요,청포도 취향,효과 있었으면 좋겠습니다


In [39]:
df['KeyBERT'] =0
for t in keywords3:
   temp = list(t.split())
   text1, text2, text3 = temp[0],  temp[1],  temp[-1]
   try:
       df.loc[(df['REVIEW'].str.contains(str(text1))) & (df['REVIEW'].str.contains(str(text2))) & (df['REVIEW'].str.contains(str(text3))) , 'KeyBERT'] = 1
   except:
       df.loc[(df['리뷰상세내용'].str.contains(str(text1))) & (df['리뷰상세내용'].str.contains(str(text2))) & (df['리뷰상세내용'].str.contains(str(text3))) , 'KeyBERT'] = 1

In [40]:
s_dt1= (datetime.today()- timedelta(0)).strftime('%Y%m%d')

excel_name = s_dt1 + '_review_data1.xlsx'
with pd.ExcelWriter(excel_name) as writer:
  df.to_excel(writer, 'sheet1')

In [ ]:
# distances[0]

### 3.모형1 : stsb-TinyBERT-L-4

In [ ]:
# nli + STS가 들어간 모델을 이용한다?
# nli이 들어간 모델을 이용한다? (X) -> n_gram에서 진행되지 않음
# stsb가 들어간 모델을 이용한다? -> OK

In [ ]:
# model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual') # 타임오버
# model = SentenceTransformer('sentence-transformers/roberta-large-nli-stsb-mean-tokens') # 타임오버

# model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens') # OK / 10분?
# model = SentenceTransformer('sentence-transformers/bert-base-nli-stsb-mean-tokens') # OK / 25분
model = SentenceTransformer('cross-encoder/stsb-TinyBERT-L-4') # OK / 1분
# model = SentenceTransformer('Muennighoff/SBERT-base-nli-stsb-v2') # OK / 26분
# model = SentenceTransformer('sentence-transformers/distilbert-base-nli-stsb-mean-tokens') # OK / 12분
# model = SentenceTransformer('sentence-transformers/stsb-roberta-base')  # OK / 22분

# model = SentenceTransformer('sentence-transformers/roberta-base-nli-stsb-mean-tokens')
# model = SentenceTransformer('sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/311 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/941 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cross-encoder_stsb-TinyBERT-L-4 were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 문서간 유사도
doc_embedding = model.encode([doc])

# 단어간 유사도 : 30분 초과시 pass
candidate_embeddings = model.encode(candidates)

In [ ]:
# KeyBERT : 기본
top_n = 15
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords2 =  max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=15, nr_candidates=20)
keywords3 = mmr(doc_embedding, candidate_embeddings, candidates, top_n=15, diversity=0.8)

In [ ]:
df_k = pd.DataFrame()
df_k['BERT'] =  keywords
df_k['BERT_MMS'] =  keywords2
df_k['BERT_MMR'] =  keywords3

In [ ]:
df_k

,BERT,BERT_MMS,BERT_MMR
0,망쳐서 조금 속상하네요,보내주셔서 만족합니다 미친,정말 촉촉하고 부드럽고
1,부드럽고 달콤합니다 항상,마시면 정말 금상첨화,가끔 먹고싶을
2,그대로 정말 촉촉하고,나름 만족합니다 적당하,크림 들어있는 케이크
3,상품 만족합니다 달지도,초코 고소함과 달달,했습니다 맛있어서 추가
4,해주셔서 만족합니다 분리,해결 해주셔서 만족합니다,보관 가능한
5,보고 언박싱 하게하려고,망쳐서 조금 속상하네요,지방 락토프리 가볍게
6,유통기간 넉넉하고 포장,부드럽고 달콤합니다 항상,않고 적당해요 정확한
7,언박싱 하게하려고 합니다,그대로 정말 촉촉하고,차원 가격 싸고
8,고소함과 달달 적당해서,상품 만족합니다 달지도,감사합니다 항상 감사합니다
9,냄새 지독하지만 만족합니다,보고 언박싱 하게하려고,바로 받아서


In [ ]:

# 요약모듈
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# article_text1 = listToString(df[(df.KeyBERT==1)& (df.PRODUCT_GROUP==1)]['REVIEW'].sample(frac = 1).to_list())
# article_text2 = listToString(df[(df.KeyBERT==1)& (df.DELIVERY_GROUP==1)]['REVIEW'].sample(frac = 1).to_list())

In [ ]:
# 내용요약 모듈
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

input_ids = tokenizer([WHITESPACE_HANDLER(article_text1)], return_tensors="pt", padding="max_length", truncation=True, max_length=512)["input_ids"]
output_ids = model.generate(input_ids=input_ids, max_length=84, no_repeat_ngram_size=2, num_beams=20)[0]
summary1 = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

input_ids = tokenizer([WHITESPACE_HANDLER(article_text2)], return_tensors="pt", padding="max_length", truncation=True, max_length=512)["input_ids"]
output_ids = model.generate(input_ids=input_ids, max_length=84, no_repeat_ngram_size=2, num_beams=20)[0]
summary2 = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)


/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


NameError: ignored

In [ ]:
summary1

In [ ]:
summary2

In [ ]:
article_text1

In [ ]:
article_text2

### 4.BERTopic 기반, 대표단어 추출

In [ ]:
okt = Okt()

tokenized_doc = okt.pos(df['REVIEW'][1])
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') | (word[1] == 'Verb') | (word[1] == 'Adjective')])

n_gram_range = (1, 1)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates_ = count.get_feature_names_out()


In [ ]:
max_sum_sim(model.encode([df['REVIEW'][1]]), candidate_embeddings, candidates_, top_n=3, nr_candidates=20)

In [ ]:
df['REVIEW'][1]

### ~~9.KoGPT-2 : 문장 생성~~

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

In [ ]:
sent = '검은콩 두유 구매시, 당의 정도는'

input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)


In [ ]:
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

In [ ]:
tokenizer.decode(output_ids)

In [ ]:
# sent = '근육이 커지기 위해서는'
sent = '검은콩 선택 요소에 당과 관련된 부분을 확인하고자 하는 이유는?'
input_ids = tokenizer.encode(sent)

import random

while len(input_ids) < 50:
    output = model(np.array([input_ids]))
    top5 = tf.math.top_k(output.logits[0, -1], k=5)
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

tokenizer.decode(input_ids)

In [ ]:
!pip install torch --upgrade -qqq
!pip install pororo -qqq
!pip install emoji -qqq
!pip install torchvision --upgrade  -qqq

In [ ]:
from pororo import Pororo
# import tensorflow as tf
zsl = Pororo(task="zero-topic", lang="ko")

In [ ]:
df.tail(2)

In [ ]:
zls_list = list(df.REVIEW)

In [ ]:
# category_= ['맛이  달아서', '맛이 달지 않아서', '영양성분이 부족해서',  '원료 함량이 낮아서', '원료가 안전하지 않아서',  '맛이 진하지 않아서' ]

category_=['맛이 뛰어난', '농도가 적절한', '칼로리 걱정이 덜한'
          ,'달지 않은', '내가 좋아하는 맛' ,'다양한 맛을 즐길 수 있는'
          ,'좋은 원재료를 사용하는','첨가물이 없는','기능성 성분이 우수한'
          ,'포만감을 주는' ,'먹기에 편리한', '맛과 영양의 밸런스가 잘 맞는'
          ,'체중관리 / 다이어트에 좋은' ,'품질관리가 잘되는 ']


In [ ]:
zls_output = []
for t_ in zls_list[:3]:
    try:
      temp = zsl(  "''" + str(t_) + "''",  category_ )
      zls_output.append(temp)
    except:
      zls_output.append('')

In [ ]:
category_

In [ ]:
zsl("맛있어요~ 늘 잘 먹고 있습니다",  category_ )

In [ ]:
zsl('''역시나 단맛이 없어 좀 밍밍한듯하지만 담백한 고소함이 느껴져서 좋네요''',  category_ )

In [ ]:
zsl('''배송 빠르고 좋아요''', category_ )

In [ ]:
zsl('''칼로리 걱정 없고 영양챙기고 좋아요''', category_ )

In [ ]:
zsl('''맛있어요 저당으로도 나와줬으면 좋겠네요''', category_ )

In [ ]:
zsl('''좀 덜 달지만 맛있어요!''', category_ )

In [ ]:
zsl('''맛있어요 담백하고 고소하고 달지않아 좋네요~단백질 함유량도 꽤 되니 더 ㅈᆢㅎ아요''', category_ )